In [1]:
%load_ext autoreload
%autoreload 2
# TODO: Use this to profile: https://stackoverflow.com/questions/45893768/how-do-i-find-out-what-parts-of-my-code-are-inefficient-in-python
%load_ext line_profiler

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import holidays
import dataclasses
import datetime
import pprint
import statistics

from src.data_generation import *
from src.data_generation_helpers import *
from src.serialization_lib import *
from src.data_types import *
from datetime import timedelta
from enum import Enum, auto
from typing import List, Optional, Tuple, Set
from pandas.tseries.offsets import BDay
pd.set_option('display.max_colwidth', None)

pp = pprint.PrettyPrinter(indent=4)

In [3]:
API_KEY = 'cfg2wsKZrVNuYBJpETAs'
DEVELOPMENT = False

if DEVELOPMENT:
    daily_metrics = pd.read_csv('SHARADAR-DAILY.csv')
    daily_prices = pd.read_csv('SHARADAR-SEP.csv')
else:
    daily_metrics = pd.read_csv('SHARADAR_DAILY_3_9ffd00fad4f19bbdec75c6e670d3df83.csv')
    daily_prices = pd.read_csv('SHARADAR_SEP_2_0bd2000858d1d8d1f48d4cdea5f8c9e2.csv')

In [4]:
d1 = daily_metrics.copy()

d2 = daily_prices[['ticker', 'date','closeadj']]
d2.rename(columns={'closeadj': 'price'}, inplace=True)

daily_data = d1.merge(d2, on=['date', 'ticker'], how='inner')
daily_data

/Users/olshansky/.pyenv/versions/3.7.7/envs/tip_env/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ticker,date,lastupdated,ev,evebit,evebitda,marketcap,pb,pe,ps,price
0,A,2020-12-18,2020-12-18,37525.1,40.8,30.6,36607.1,7.5,50.9,6.9,118.928
1,AA,2020-12-18,2020-12-18,4896.3,-43.0,8.9,4092.3,1.2,-8.7,0.4,22.010
2,AACQ,2020-12-18,2020-12-18,943.3,NaN,NaN,944.6,188.9,NaN,NaN,10.430
3,AAIC,2020-12-18,2020-12-18,817.6,-15.8,-15.8,125.0,0.5,-2.3,5.3,3.740
4,AAIIQ,2020-12-18,2020-12-18,-2.8,0.5,0.8,0.4,0.0,-0.1,0.0,0.100
...,...,...,...,...,...,...,...,...,...,...,...
14551055,GSTCQ,2011-01-04,2021-05-30,235.6,3.4,3.0,218.6,1.4,-575.3,5.0,4.340
14551056,MHRCQ,2011-01-04,2021-05-30,471.6,-29.7,-64.3,419.2,6.0,-19.3,13.6,7.230
14551057,EPRSQ,2011-01-03,2021-05-30,153.1,187.2,8.6,155.8,2.1,74.4,2.9,105.000
14551058,GSTCQ,2011-01-03,2021-05-30,242.7,3.5,3.1,225.7,1.4,-593.9,5.1,4.480


In [7]:
# Prepare Inputs for Base + Test
INITIAL_PORTFOLIO_VALUE = 10000
PORTFOLIO_SIZE = 30
REBALANCE_DAYS = 90

BASE_METRIC = EvaluationMetric.EV_EBIT
TEST_METRIC = EvaluationMetric.P_B
STOCKS_UNIVERSE = StockUniverse.LARGE
PORTFOLIO_WEIGHT_STRATEGY = StockBasketWeightApproach.EQUAL_WEIGHTING

# OPTIMIZATION ONLY: Sorting every time would take too long...
date_sorted_daily_data = daily_data.sort_values(by='date')
base_sorted_daily_data = sort_df_by_metric(daily_data, BASE_METRIC)
test_sorted_daily_data = sort_df_by_metric(daily_data, TEST_METRIC)

In [9]:
# for rebalance_days in [365]:
#     for portfolio_size in [10]:
for rebalance_days in [30, 90, 180, 730, 1825]:
    for portfolio_size in [5, 10, 15, 30, 60]:        
        print(rebalance_days, portfolio_size)
        df_res, df_debug = compute_dfs(
            BASE_METRIC,
            TEST_METRIC,
            STOCKS_UNIVERSE,
            PORTFOLIO_WEIGHT_STRATEGY,
            rebalance_days,
            portfolio_size,
            INITIAL_PORTFOLIO_VALUE,
            daily_data,
            date_sorted_daily_data,
            base_sorted_daily_data,
            test_sorted_daily_data)

2012-01-03
2013-01-02
2014-01-02
2015-01-02
2015-12-31
2016-12-30
2017-12-29
2018-12-31
2019-12-31
2020-12-31


In [21]:
filename = df_feather_filename('df_debug', BASE_METRIC, TEST_METRIC, REBALANCE_DAYS, PORTFOLIO_SIZE, STOCKS_UNIVERSE)
write_df_debug_to_feather(df_debug, filename)

filename = df_feather_filename('df_res', BASE_METRIC, TEST_METRIC, REBALANCE_DAYS, PORTFOLIO_SIZE, STOCKS_UNIVERSE)
write_df_res_to_feather(df_res, filename)

In [22]:
filename

'df_res:EV_EBIT_VSP_B:rebalanced_every_90:portfolio_size_30:StockUniverse.LARGE.feather'